In [1]:
from bokeh.plotting import figure, show, output_notebook
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import Point

In [2]:
region_filename = 'berkeley-s7996m-geojson.json'  # file with geojson regions
regions = gpd.read_file(region_filename)

In [3]:
data_filename = '1549304575_89608.csv'  # file with time, lat, long data
# data_filename = '1549304575_89608.csv'
df = pd.read_csv(data_filename)
df['datetime'] = pd.to_datetime(df['_time'])
df['coordinates'] = list(zip(df.lng, df.lat))  # create a coordinates pair
df['geometry'] = df['coordinates'].apply(Point)  # create POINT objects
df.drop(['_time', 'lat', 'lng', 'coordinates'], axis=1, inplace=True)  # get rid of extra crap

In [4]:
points = gpd.GeoDataFrame(df, geometry='geometry')

In [5]:
points_and_regions = gpd.sjoin(regions, points, how="inner", op='contains')
points_and_regions.drop(['geometry'], axis=1, inplace=True)  # get rid of extra crap
# print(points_and_regions)
points_and_regions.set_index(['datetime'], inplace=True)
points_and_regions.index = points_and_regions.index.tz_localize('UTC').tz_convert('America/Los_Angeles')

C:\Users\Steve\Anaconda3\lib\site-packages\geopandas\tools\sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')


In [6]:
# lol = points_and_regions.groupby(['id', pd.Grouper(freq='1H')]).count()
# print(points_and_regions.head(5))
t_index = pd.DatetimeIndex(start=points_and_regions.index.min().date(), end=points_and_regions.index.max().date()  + pd.DateOffset(1), freq='3H', tz='America/Los_Angeles')[:-1] # don't include the new day that is the last point >:O
# print(t_index)
lol = points_and_regions.groupby('id').resample('3H').count()  # TODO: doesn't count from 00:00 - counts from earliest point

print(lol.head(5))
lol=lol.unstack(level=0).fillna(0).reindex(t_index).stack('id').swaplevel(0,1).sort_index()
# lol=lol.reindex(t_index)
# lol=lol.stack('id')
# lol=lol.swaplevel(0,1).sort_index()
lol.rename({'id': 'count'}, axis='columns', inplace=True)

# print(lol.loc[('s7996m.2', slice(None)), ('count')])
print(lol.head(5))


                                    id  Subbeat  BEAT  Shape_area  Shape_len  \
id       datetime                                                              
s7996m.1 2018-06-30 15:00:00-07:00   3        3     3           3          3   
         2018-06-30 18:00:00-07:00   3        3     3           3          3   
         2018-06-30 21:00:00-07:00   7        7     7           7          7   
         2018-07-01 00:00:00-07:00   2        2     2           2          2   
         2018-07-01 03:00:00-07:00   0        0     0           0          0   

                                    index_right  
id       datetime                                
s7996m.1 2018-06-30 15:00:00-07:00            3  
         2018-06-30 18:00:00-07:00            3  
         2018-06-30 21:00:00-07:00            7  
         2018-07-01 00:00:00-07:00            2  
         2018-07-01 03:00:00-07:00            0  
                                    count  Subbeat  BEAT  Shape_area  \
id               

In [7]:
# cat = lol.loc[('s7996m.2'), ('count')]
# t_index = lol.loc[('s7996m.1'), ('count')].index
# get earliest and latested times
# t_index.tz_localize('America/Los_Angeles')
# print(cat)
# cat.reindex(t_index).fillna(0)

Here we didn't merge lol with regions to make a mega dataframe with geometry. Hopefully we can just reference the ID when using regions.

In [8]:
lol.reset_index(inplace=True)
lol.rename({'level_1': 'datetime'}, axis='columns', inplace=True)

print(lol.head(5))

lol['day_of_week'] = lol.datetime.dt.dayofweek
lol['hour'] = lol.datetime.dt.hour
# lol['date'] = lol.datetime.dt.date

# print(lol.head(10))
lol['hour_bucket'] = lol.groupby([lol.id, lol.datetime.dt.date]).cumcount()
# TODO: either make a new column of index values or figure out how to specify just the hour (bucket) by iloc
lol.set_index(['id', 'day_of_week', 'hour_bucket'], inplace=True)
lol.loc[('s7996m.2', 0, slice(None)), ('datetime', 'count','hour')]



         id                  datetime  count  Subbeat  BEAT  Shape_area  \
0  s7996m.1 2018-06-30 15:00:00-07:00    3.0      3.0   3.0         3.0   
1  s7996m.1 2018-06-30 18:00:00-07:00    3.0      3.0   3.0         3.0   
2  s7996m.1 2018-06-30 21:00:00-07:00    7.0      7.0   7.0         7.0   
3  s7996m.1 2018-07-01 00:00:00-07:00    2.0      2.0   2.0         2.0   
4  s7996m.1 2018-07-01 03:00:00-07:00    0.0      0.0   0.0         0.0   

   Shape_len  index_right  
0        3.0          3.0  
1        3.0          3.0  
2        7.0          7.0  
3        2.0          2.0  
4        0.0          0.0  


datetime  count  hour
id       day_of_week hour_bucket                                       
s7996m.2 0           0           2018-07-02 00:00:00-07:00    0.0     0
                     1           2018-07-02 03:00:00-07:00    0.0     3
                     2           2018-07-02 06:00:00-07:00    1.0     6
                     3           2018-07-02 09:00:00-07:00    3.0     9
                     4           2018-07-02 12:00:00-07:00    2.0    12
                     5           2018-07-02 15:00:00-07:00    5.0    15
                     6           2018-07-02 18:00:00-07:00    3.0    18
                     7           2018-07-02 21:00:00-07:00    2.0    21
                     0           2018-07-09 00:00:00-07:00    0.0     0
                     1           2018-07-09 03:00:00-07:00    0.0     3
                     2           2018-07-09 06:00:00-07:00    1.0     6
                     3           2018-07-09 09:00:00-07:00    2.0     9
                     4           2018-07-09 12:00:00-07:00    4.0    12
                     5           2018-07-09 15:00:00-07:00    4.0    15
                     6           2018-07-09 18:00:00-07:00    3.0    18
                     7           2018-07-09 21:00:00-07:00    0.0    21
                     0           2018-07-16 00:00:00-07:00    0.0     0
                     1           2018-07-16 03:00:00-07:00    0.0     3
                     2           2018-07-16 06:00:00-07:00    3.0     6
                     3           2018-07-16 09:00:00-07:00    3.0     9
                     4           2018-07-16 12:00:00-07:00    2.0    12
                     5           2018-07-16 15:00:00-07:00    1.0    15
                     6           2018-07-16 18:00:00-07:00    2.0    18
                     7           2018-07-16 21:00:00-07:00    2.0    21
                     0           2018-07-23 00:00:00-07:00    0.0     0
                     1           2018-07-23 03:00:00-07:00    0.0     3
                     2           2018-07-23 06:00:00-07:00    3.0     6
                     3           2018-07-23 09:00:00-07:00    5.0     9
                     4           2018-07-23 12:00:00-07:00    5.0    12
                     5           2018-07-23 15:00:00-07:00    3.0    15
...                                                    ...    ...   ...
                     2           2018-09-03 06:00:00-07:00    1.0     6
                     3           2018-09-03 09:00:00-07:00    6.0     9
                     4           2018-09-03 12:00:00-07:00    6.0    12
                     5           2018-09-03 15:00:00-07:00    1.0    15
                     6           2018-09-03 18:00:00-07:00    4.0    18
                     7           2018-09-03 21:00:00-07:00    0.0    21
                     0           2018-09-10 00:00:00-07:00    0.0     0
                     1           2018-09-10 03:00:00-07:00    1.0     3
                     2           2018-09-10 06:00:00-07:00    1.0     6
                     3           2018-09-10 09:00:00-07:00    5.0     9
                     4           2018-09-10 12:00:00-07:00    5.0    12
                     5           2018-09-10 15:00:00-07:00    6.0    15
                     6           2018-09-10 18:00:00-07:00    6.0    18
                     7           2018-09-10 21:00:00-07:00    1.0    21
                     0           2018-09-17 00:00:00-07:00    0.0     0
                     1           2018-09-17 03:00:00-07:00    0.0     3
                     2           2018-09-17 06:00:00-07:00    3.0     6
                     3           2018-09-17 09:00:00-07:00    2.0     9
                     4           2018-09-17 12:00:00-07:00    6.0    12
                     5           2018-09-17 15:00:00-07:00    2.0    15
                     6           2018-09-17 18:00:00-07:00    9.0    18
                     7           2018-09-17 21:00:00-07:00    0.0    21
                     0           2018-09-24 00:00:00-07:00    1.0     0
                  

In [9]:
lol.loc[('s7996m.1', 0, slice(None)), ('datetime', 'count', 'hour')]

datetime  count  hour
id       day_of_week hour_bucket                                       
s7996m.1 0           0           2018-07-02 00:00:00-07:00    0.0     0
                     1           2018-07-02 03:00:00-07:00    1.0     3
                     2           2018-07-02 06:00:00-07:00    8.0     6
                     3           2018-07-02 09:00:00-07:00    3.0     9
                     4           2018-07-02 12:00:00-07:00    3.0    12
                     5           2018-07-02 15:00:00-07:00    6.0    15
                     6           2018-07-02 18:00:00-07:00    7.0    18
                     7           2018-07-02 21:00:00-07:00    3.0    21
                     0           2018-07-09 00:00:00-07:00    1.0     0
                     1           2018-07-09 03:00:00-07:00    0.0     3
                     2           2018-07-09 06:00:00-07:00    3.0     6
                     3           2018-07-09 09:00:00-07:00    4.0     9
                     4           2018-07-09 12:00:00-07:00    2.0    12
                     5           2018-07-09 15:00:00-07:00   12.0    15
                     6           2018-07-09 18:00:00-07:00    9.0    18
                     7           2018-07-09 21:00:00-07:00    2.0    21
                     0           2018-07-16 00:00:00-07:00    4.0     0
                     1           2018-07-16 03:00:00-07:00    1.0     3
                     2           2018-07-16 06:00:00-07:00    4.0     6
                     3           2018-07-16 09:00:00-07:00    5.0     9
                     4           2018-07-16 12:00:00-07:00    8.0    12
                     5           2018-07-16 15:00:00-07:00    3.0    15
                     6           2018-07-16 18:00:00-07:00    3.0    18
                     7           2018-07-16 21:00:00-07:00    6.0    21
                     0           2018-07-23 00:00:00-07:00    6.0     0
                     1           2018-07-23 03:00:00-07:00    0.0     3
                     2           2018-07-23 06:00:00-07:00    7.0     6
                     3           2018-07-23 09:00:00-07:00    3.0     9
                     4           2018-07-23 12:00:00-07:00    2.0    12
                     5           2018-07-23 15:00:00-07:00    4.0    15
...                                                    ...    ...   ...
                     2           2018-09-03 06:00:00-07:00    1.0     6
                     3           2018-09-03 09:00:00-07:00   11.0     9
                     4           2018-09-03 12:00:00-07:00   16.0    12
                     5           2018-09-03 15:00:00-07:00   13.0    15
                     6           2018-09-03 18:00:00-07:00   10.0    18
                     7           2018-09-03 21:00:00-07:00   15.0    21
                     0           2018-09-10 00:00:00-07:00    0.0     0
                     1           2018-09-10 03:00:00-07:00    0.0     3
                     2           2018-09-10 06:00:00-07:00    8.0     6
                     3           2018-09-10 09:00:00-07:00    3.0     9
                     4           2018-09-10 12:00:00-07:00    2.0    12
                     5           2018-09-10 15:00:00-07:00    5.0    15
                     6           2018-09-10 18:00:00-07:00   10.0    18
                     7           2018-09-10 21:00:00-07:00   15.0    21
                     0           2018-09-17 00:00:00-07:00    4.0     0
                     1           2018-09-17 03:00:00-07:00    1.0     3
                     2           2018-09-17 06:00:00-07:00    8.0     6
                     3           2018-09-17 09:00:00-07:00   11.0     9
                     4           2018-09-17 12:00:00-07:00    9.0    12
                     5           2018-09-17 15:00:00-07:00   10.0    15
                     6           2018-09-17 18:00:00-07:00    7.0    18
                     7           2018-09-17 21:00:00-07:00    8.0    21
                     0           2018-09-24 00:00:00-07:00    6.0     0
                  

In [10]:
c = lol.loc[(slice(None), 6, 0), ('count')]
print(len(c))
print(c)
a = c.index.get_level_values(0).values
b = c.values
print(a, b)


325
id         day_of_week  hour_bucket
s7996m.1   6            0               2.0
                        0               3.0
                        0               3.0
                        0               3.0
                        0               6.0
                        0               9.0
                        0               4.0
                        0               5.0
                        0              10.0
                        0               4.0
                        0              12.0
                        0               6.0
                        0               7.0
s7996m.10  6            0               1.0
                        0               1.0
                        0               1.0
                        0               1.0
                        0               0.0
                        0               0.0
                        0               0.0
                        0               2.0
                        0           

In [11]:
regions.sort_values(by=['id'], inplace=True)

In [12]:
hours = [str(x).zfill(2) + ':00' for x in list(lol.loc[(slice(None), slice(None), slice(None)), ('hour')].unique())]
hours

['15:00', '18:00', '21:00', '00:00', '03:00', '06:00', '09:00', '12:00']

In [13]:
lol.loc[('s7996m.2', slice(None), slice(None)), ('count', 'hour')]

count  hour
id       day_of_week hour_bucket             
s7996m.2 5           0              0.0    15
                     1              2.0    18
                     2              2.0    21
         6           0              0.0     0
                     1              0.0     3
                     2              1.0     6
                     3              6.0     9
                     4              5.0    12
                     5              8.0    15
                     6              4.0    18
                     7              2.0    21
         0           0              0.0     0
                     1              0.0     3
                     2              1.0     6
                     3              3.0     9
                     4              2.0    12
                     5              5.0    15
                     6              3.0    18
                     7              2.0    21
         1           0              0.0     0
                     1              0.0     3
                     2              3.0     6
                     3              0.0     9
                     4              0.0    12
                     5              2.0    15
                     6              6.0    18
                     7              0.0    21
         2           0              0.0     0
                     1              0.0     3
                     2              0.0     6
...                                 ...   ...
                     0              1.0     0
                     1              0.0     3
                     2              1.0     6
                     3              0.0     9
                     4              4.0    12
                     5              3.0    15
                     6              1.0    18
                     7              1.0    21
         3           0              1.0     0
                     1              0.0     3
                     2              4.0     6
                     3              3.0     9
                     4              7.0    12
                     5              1.0    15
                     6              5.0    18
                     7              5.0    21
         4           0              1.0     0
                     1              0.0     3
                     2              5.0     6
                     3              2.0     9
                     4              3.0    12
                     5              4.0    15
                     6              8.0    18
                     7              1.0    21
         5           0              1.0     0
                     1              0.0     3
                     2              1.0     6
                     3              5.0     9
                     4              8.0    12
                     5              8.0    15

[729 rows x 2 columns]

In [14]:
lol.loc[(slice(None), 0, 0), ('count')]

id         day_of_week  hour_bucket
s7996m.1   0            0               0.0
                        0               1.0
                        0               4.0
                        0               6.0
                        0               0.0
                        0               1.0
                        0               0.0
                        0               2.0
                        0               1.0
                        0              13.0
                        0               0.0
                        0               4.0
                        0               6.0
s7996m.10  0            0               0.0
                        0               0.0
                        0               1.0
                        0               2.0
                        0               3.0
                        0               0.0
                        0               0.0
                        0               0.0
                        0               

In [15]:
lol.loc[(slice(None), slice(None), slice(None)), ('count')].max(level=0).max()

55.0

In [16]:
sorted(lol.loc[(slice(None), slice(None), slice(None)), ('hour')].unique())

[0, 3, 6, 9, 12, 15, 18, 21]

In [17]:
aa, bb =lol.loc[('s7996m.2', 0, slice(None)), ('hour', 'count')].values.transpose()
aa

array([ 0.,  3.,  6.,  9., 12., 15., 18., 21.,  0.,  3.,  6.,  9., 12.,
       15., 18., 21.,  0.,  3.,  6.,  9., 12., 15., 18., 21.,  0.,  3.,
        6.,  9., 12., 15., 18., 21.,  0.,  3.,  6.,  9., 12., 15., 18.,
       21.,  0.,  3.,  6.,  9., 12., 15., 18., 21.,  0.,  3.,  6.,  9.,
       12., 15., 18., 21.,  0.,  3.,  6.,  9., 12., 15., 18., 21.,  0.,
        3.,  6.,  9., 12., 15., 18., 21.,  0.,  3.,  6.,  9., 12., 15.,
       18., 21.,  0.,  3.,  6.,  9., 12., 15., 18., 21.,  0.,  3.,  6.,
        9., 12., 15., 18., 21.,  0.,  3.,  6.,  9., 12., 15., 18., 21.])

In [18]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
print(lol.loc[('s7996m.20', 5, 6), 'datetime'])
x=lol.loc[('s7996m.20', 5, 6), 'count'].view('int64').values
print(type(x))
print(x)
x=sorted(x)
x=np.asarray(x)
print('meow')
y=lol.loc[('s7996m.20', 5, 6), 'count'].values
print(y)
regr = linear_model.LinearRegression()
# fit on all the data except the last point
regr.fit(x[:-1].reshape(-1, 1), y[:-1])
# compute what the linear model would show on all points
y_linear = regr.predict(x.reshape(-1, 1))
print(y_linear)
from matplotlib import pyplot as plt
plt.scatter(x,y)
plt.scatter(x,y_linear)

plt.show()


id         day_of_week  hour_bucket
s7996m.20  5            6             2018-07-07 18:00:00-07:00
                        6             2018-07-14 18:00:00-07:00
                        6             2018-07-21 18:00:00-07:00
                        6             2018-07-28 18:00:00-07:00
                        6             2018-08-04 18:00:00-07:00
                        6             2018-08-11 18:00:00-07:00
                        6             2018-08-18 18:00:00-07:00
                        6             2018-08-25 18:00:00-07:00
                        6             2018-09-01 18:00:00-07:00
                        6             2018-09-08 18:00:00-07:00
                        6             2018-09-15 18:00:00-07:00
                        6             2018-09-22 18:00:00-07:00
Name: datetime, dtype: datetime64[ns, America/Los_Angeles]
<class 'numpy.ndarray'>
[4618441417868443648 4625478292286210048 4624070917402656768
 4622382067542392832 4625759767262920704 46263227172

C:\Users\Steve\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


<Figure size 640x480 with 1 Axes>

In [19]:
sorted(lol.loc[('s7996m.20', 0, 0), ('count')].view('int64').values)

C:\Users\Steve\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


[4611686018427387904,
 4613937818241073152,
 4613937818241073152,
 4617315517961601024,
 4617315517961601024,
 4617315517961601024,
 4618441417868443648,
 4619567317775286272,
 4620693217682128896,
 4620693217682128896,
 4622382067542392832,
 4624070917402656768,
 4625478292286210048]

In [38]:
rl=lol.loc[(slice(None), slice(None), slice(None)), slice(None)].index.get_level_values(0).unique()
rl

Index(['s7996m.1', 's7996m.10', 's7996m.11', 's7996m.12', 's7996m.13',
       's7996m.14', 's7996m.15', 's7996m.16', 's7996m.17', 's7996m.18',
       's7996m.19', 's7996m.2', 's7996m.20', 's7996m.21', 's7996m.22',
       's7996m.23', 's7996m.24', 's7996m.25', 's7996m.3', 's7996m.4',
       's7996m.5', 's7996m.6', 's7996m.7', 's7996m.8', 's7996m.9'],
      dtype='object', name='id')

In [45]:
# change alpha based on id 
type(rl)
q=rl.tolist()
[1.0 if x == 's7996m.14' else 0.0 for x in rl]
[1.0 for x in rl]

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]